In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[]

In [15]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [17]:
train_path = './dataset/train'
valid_path = './dataset/val'
test_path = './dataset/test'

breeds = [
    "Holstein_cow", "Jersey_cow", "Angus_cow", "Brahman_cow", "Hereford_cow",
    "Simmental_cow", "Limousin_cow", "Guernsey_cow", "Charolais_cow", "Ayrshire_cow"
]

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
                .flow_from_directory(directory=train_path, target_size=(224,224), classes=breeds, batch_size = 10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
                .flow_from_directory(directory=valid_path, target_size=(224,224), classes=breeds, batch_size = 10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
                .flow_from_directory(directory=test_path, target_size=(224,224), classes=breeds, batch_size = 10, shuffle=False)

Found 2678 images belonging to 10 classes.
Found 912 images belonging to 10 classes.
Found 906 images belonging to 10 classes.


In [ ]:
# Load results from all models into a dictionary
results = {}
with open('results_resnet50.pkl', 'rb') as f:
    results['ResNet50'] = pickle.load(f)
with open('results_densenet121.pkl', 'rb') as f:
    results['DenseNet121'] = pickle.load(f)
with open('mobilenetv3_results.pkl', 'rb') as f:
    results['MobileNetV3Large'] = pickle.load(f)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Visualize training loss and accuracy for each model
for model_name, res in results.items():
    history = res['history']
    plt.figure(figsize=(12, 4))
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc')
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
# Display confusion matrix for each model on validation set
for model_name, res in results.items():
    y_true = np.argmax(res['val_labels'], axis=1)
    y_pred = np.argmax(res['val_preds'], axis=1)
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=breeds)
    plt.figure(figsize=(8, 8))
    disp.plot(cmap='Blues', xticks_rotation=45)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.show()

In [ ]:
# Evaluate all models on the test set
for model_name, res in results.items():
    model = None
    if model_name == 'ResNet50':
        model = resnet50_model
    elif model_name == 'DenseNet121':
        model = densenet121_model
    elif model_name == 'MobileNetV3Large':
        model = mobilenetv3_model
    if model is not None:
        test_imgs, test_labels = next(test_batches)
        test_preds = model.predict(test_imgs)
        test_acc = np.mean(np.argmax(test_preds, axis=1) == np.argmax(test_labels, axis=1))
        test_mAP = mean_average_precision(test_labels, test_preds)
        print(f"{model_name} Test Accuracy: {test_acc:.4f}, Test mAP: {test_mAP:.4f}")

In [ ]:
# Final conclusion
print("\nFinal Conclusion:")
summary = []
for model_name, res in results.items():
    summary.append({
        'Model': model_name,
        'Training Time (s)': res['training_time_sec'],
        'Validation Accuracy': res['val_accuracy'],
        'Validation mAP': res['val_mAP']
    })
df_summary = pd.DataFrame(summary)
display(df_summary.sort_values(by=['Validation Accuracy', 'Validation mAP'], ascending=False))
print("Based on the above metrics (accuracy, mAP, training time), select the model with the best trade-off for your application. Typically, the model with the highest accuracy and mAP, and reasonable training time, is preferred.")

FileNotFoundError: [Errno 2] No such file or directory: 'results_resnet50.pkl'